In [24]:
import os
from google.colab import drive
drive.mount('/content/drive2')
path = '/content/drive2/Shared drives/Emotion Classification Through Deep Learning Drive/fer2013/FaceImages'
os.chdir(path)

from PIL import Image
import cv2
#Create read path <rdPath> and write path <wrPath>
rdPath = path + '/Unresized'
wrPath = path + '/Resized'
#Desired pixel
desire_size = (48,48)
print(rdPath)
print(wrPath)

Drive already mounted at /content/drive2; to attempt to forcibly remount, call drive.mount("/content/drive2", force_remount=True).


In [0]:
def data_reader(input_dir,limit):
    file_list = []
    length = 0
    for file in os.scandir(input_dir):
            if length >= limit:
                break
            if file.is_file():
                file_list.append(file.name)
                length += 1
    return file_list

def FacialRecognition(files,rdPath,wrPath,index):
    print('Read',files[0])
    
    for f in files:
        fullPath = os.path.join(rdPath,f)
        img = cv2.imread(fullPath)
        #Make it gray
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        #Find faces
        faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
        faces = faceCascade.detectMultiScale(
            gray,
            scaleFactor=1.3,
            minNeighbors=3,
            minSize=(48, 48)
        )
        #Create faces rectangle and resize it
        for (x, y, w, h) in faces:
            cv2.rectangle(gray, (x, y), (x + w, y + h), (0, 255, 0), 2)
            roi_color = gray[y:y + h, x:x + w]
            img_pil = Image.fromarray(roi_color)
            new_img = img_pil.resize(desire_size)
            #N
            wrName = f.split("_")[0].capitalize() + '_%d.png' % (index)
            new_img.save(os.path.join(wrPath, wrName),'PNG')
            index+=1
        #Remove a file
        os.remove(fullPath)
        return index


In [0]:
index = 0
#Run until rdPath is empty
while len(os.listdir(rdPath)) != 0:
    files = data_reader(rdPath,200)
    index = FacialRecognition(files,rdPath,wrPath,index)